In [1]:
import numpy as np
import pandas as pd
from sklearn.externals.six.moves import xrange
from sklearn.mixture import GMM
from sklearn.externals import joblib

#from sklearn.decomposition import PCA
#import matplotlib.pyplot as plt


In [2]:
train=pd.read_csv('data_final.csv',header=0)

#Separate out X_train and y_train
train = train[['lr','rl','Lr','Rl','ll','rr','l','r','L','R','Space','Enter','cpm','y']]

X=train
y=train

X=X.drop(['y'],axis=1) #remove the output label column

y=y.drop(['lr','rl','Lr','Rl','ll','rr','l','r','L','R','Space','Enter','cpm'],axis=1) #remove all the columns except the output label
#y=train.drop(['lr','rl','lR','Lr','rL','Rl','lL','rR','ll','rr','l','r','L','R','Space','Backspace','Enter','cpm'],axis=1) #remove all the columns except the output label


X=X.values
y=y.values

msk = np.random.rand(len(X)) < 0.8
X_train = X[msk]
y_train = y[msk]

X_test = X[~msk]
y_test = y[~msk]


#X_train=X
#y_train=y


In [3]:
#GMM 
n_classes=len(np.unique(y_train)) # should be 5/6
classifiers = dict((covar_type, GMM(n_components=n_classes,
                    covariance_type=covar_type, init_params='wc', n_iter=40))
                   for covar_type in ['tied'])
                       
n_classifiers = len(classifiers)

#print X_train.shape
#print X_train[0]


for index, (name, classifier) in enumerate(classifiers.items()):
    
    #Initializing the Means of the GMM manually
#    print("X_train.shape:", X_train.shape)
#    print("y_train.shape:", y_train.shape)
    y_train = y_train.flatten()


#    z = y_train == (0+1)
#    print("z.shape", z.shape)
#    print("X_train[y_train == (0+1)].shape:", X_train[y_train == (0+1)].shape)
    classifier.means_ = np.array([X_train[y_train == (i)].mean(axis=0)
                                  for i in xrange(n_classes)])
#    print("classifier.means_:", classifier.means_)                                  
    #Fitting the training data
    classifier.fit(X_train)
#    #Predictions
    y_train_pred = classifier.predict(X_train)   
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) * 100
    print "The accuracy in training is ",train_accuracy,"\n"    

    joblib.dump(classifier,'GMMclassifier.pkl')

The accuracy in training is  67.8955453149 



In [4]:
GMM_classifier=joblib.load('GMMclassifier.pkl')
y_test_pred=GMM_classifier.predict(X_test)
test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
   
print "The accuracy in test set is ",test_accuracy,"\n"    


The accuracy in test set is  70.7865168539 



In [17]:


import numpy as np
import pandas as pd
import csv
from date_time import timevalue 
from process import categorize
from process import lr_holdtime


fr=open('test_ankesh.txt','rb') #test file of person to identify
info=fr.readlines() 
fw=open('info.csv','wb')

in_txt=csv.reader(info,delimiter='\t')
out_csv=csv.writer(fw)
out_csv.writerows(in_txt)

fr.close()
fw.close()
columns=['Event_Type','Key_Code','Shift','Alt','Control','Time']
df=pd.read_csv('info.csv')
if len(df.columns)>6:
    for i in range(len(df.columns)-1,5,-1):
        df=df.drop(df.columns[[i]],axis=1)
df.columns=columns;

del_rows=[10,38,39,40,41,165,161,163,162,164,166]
for i in del_rows:
    df=df[df.Key_Code!=i]
df=df[df.Alt==False]
df=df[df.Control==False]
df=df.drop('Alt',axis=1)
df=df.drop('Control',axis=1)

dates=df.Time.tolist()
dates=pd.Series(timevalue(dates))
df=df.reset_index()
df.Time=dates
df=df.drop('index',axis=1)




final_features=pd.DataFrame(columns=('lr','rl','lR','Lr','rL','Rl','lL','rR','ll','rr','l','r','L','R','Space','Enter',
                                     'Backspace','cpm'))

label=2

#initial value
index=-1
count=0

while index<len(df)-1:
    
    tmp_df=pd.DataFrame(columns=('Event_Type','Key_Code','Shift','Time'))
    feature_list=np.zeros((2,11))
    mean_latencies=np.zeros(11)
    i=0    
    index=index+1
    
    tmp=[df.Event_Type[index],df.Key_Code[index],df.Shift[index],df.Time[index]]
    tmp_df.loc[i]=tmp

    i=i+1

    while 1:
        
        if index>len(df)-1:
            break
        
        tmp=[df.Event_Type[index],df.Key_Code[index],df.Shift[index],df.Time[index]]
        if tmp[3]-tmp_df.Time[i-1]>10:
            break
        
        if tmp[3]-tmp_df.Time[0]>30:
            break
        
        tmp_df.loc[i]=tmp
        i=i+1
        index=index+1
            
    
    
    if len(tmp_df.Time)<50:
        continue
    
   ############################################################################
    #Calculating latencies of the different features

    
    
    for j in range(0,len(tmp_df.Time)-1):
        
        flag=0
        
        if tmp_df.Event_Type[j]=='KeyDown':
            for k in range(j+1,len(tmp_df.Time)):
                if tmp_df.Event_Type[k]=='KeyDown':
                    flag=1
                    chars=[tmp_df.Key_Code[j],tmp_df.Shift[j],tmp_df.Key_Code[k],tmp_df.Shift[k]]
                    latency=tmp_df.Time[k]-tmp_df.Time[j]
                    break
                
        if flag==1 and latency<1.5:           
            feature=categorize(chars)
            feature_list[0][feature]=feature_list[0][feature]+latency
            feature_list[1][feature]=feature_list[1][feature]+1
    
    mean_latencies=feature_list[0]/feature_list[1]
   ############################################################################            
    
    #Calculating hold times and number of backspaces and cpm
    feature_list_holdtime=np.zeros((2,7))
    backspaces=0
    characters=0
    cpm=float('NaN')
    backspace_per_character=float('NaN')
    
    for j in range(0,len(tmp_df.Time)-1):
        
        flag=0
        
        if tmp_df.Event_Type[j]=='KeyDown':
            key=tmp_df.Key_Code[j]    
            for k in range(j+1,len(tmp_df.Time)):
                if tmp_df.Key_Code[k]==key:
                    flag=1
                    chars=[key,tmp_df.Shift[j]]
                    holdtime=tmp_df.Time[k]-tmp_df.Time[j]
                    break
        
            characters=characters+1
            
            if key==9:
                backspaces=backspaces+1
                
        if flag==1 and holdtime<1.5:
            feature=lr_holdtime(chars)
            feature_list_holdtime[0][feature]=feature_list_holdtime[0][feature]+holdtime
            feature_list_holdtime[1][feature]=feature_list_holdtime[1][feature]+1
        
    mean_holdtime=feature_list_holdtime[0]/feature_list_holdtime[1]
        
    if characters>50:
        cpm=characters/(tmp_df.Time[len(tmp_df.Time)-1]-tmp_df.Time[0])*60
        backspace_per_character=float(backspaces)/characters
    
   ############################################################################
    
    mean_latencies=np.delete(mean_latencies,0,0)
    mean_holdtime=np.delete(mean_holdtime,0,0)    
    total_features=np.append(mean_latencies,mean_holdtime)
    total_features=np.append(total_features,backspace_per_character)
    total_features=np.append(total_features,cpm)
    
        
        
    
    final_features.loc[count]=total_features
    count=count+1    
    
    print count,'-',index, '-',len(tmp_df.Time)
    

           
new_final_features=final_features.dropna(how='all')
new_final_features=new_final_features.reset_index()
tmp=np.empty(len(new_final_features))
tmp.fill(label)

test=new_final_features.fillna(final_features.mean())
test=test.drop('index',axis=1)
test1=test[['lr','rl','Lr','Rl','ll','rr','l','r','L','R','Space','Enter','Backspace','cpm']]

#test1.to_csv('test_sinha.csv',index=False)
test.to_csv('test_ankesh.csv',index=False)

1 - 243 - 244
2 - 395 - 152
3 - 596 - 201
4 - 868 - 232
5 - 1062 - 194
6 - 1398 - 336
7 - 1597 - 199
8 - 1932 - 335
9 - 2182 - 250


In [5]:
import pandas as pd
from sklearn.externals import joblib


#Feature Extraction part
#Output - array of input (n,N_features)
X_test=pd.read_csv('test_soubhagya.csv',header=0)  #test file


X_test=X_test[['lr','rl','Lr','Rl','ll','rr','l','r','L','R','Space','Enter','cpm']]
X_test=X_test.values
GMM_classifier=joblib.load('GMMclassifier.pkl')
y_test=GMM_classifier.predict(X_test)

Person={0:'Daivik',1:'soubhagya',2:'Abhishek',3:'Abhinav',4:'srichandra',5:'Ankesh'}

y_test_length=len(y_test)
y_=y_test.tolist()
mode=max(set(y_), key=y_.count)
frequency=y_.count(mode)
accuracy=float(frequency)/len(y_)*100

if  accuracy > 50:
    print "The Person identified typing is ", Person[mode],"with",round(accuracy,2),'% accuracy'
else:
    print "Impostor!"
        
    

The Person identified typing is  soubhagya with 84.21 % accuracy
